# Custom Tools Demo
<img src="https://eu-images.contentstack.com/v3/assets/blt6b0f74e5591baa03/blt5d5323f706ef288f/637c0195c162df49beaae102/Untitled_design_(77).png?width=1280&auto=webp&quality=95&format=jpg&disable=upscale" width=500 />

In [0]:
%pip install --upgrade --quiet langchain-core databricks-vectorsearch langchain-community youtube_search wikipedia
#probably don't need the youtbue or wikipedia libaries, but hey whatever

dbutils.library.restartPython()

## Simulating a few API Calls
We're going to build some tools around some API calls. For that we'll need to find a few good analogs for things like time series data
<br/>
Simulate time series retrieval<br/>
	•	REST API for weather forecasts and historical data.<br/>
	•	Simulates querying sensor data or time series like in Cognite.<br/>
GET https://api.open-meteo.com/v1/forecast?latitude=51.0478&longitude=114.0593&hourly=temperature_2m<br/>
<br/>
Simulate asset metadata catalog<br/>
	•	Can simulate fetching metadata about real-world locations (assets).<br/>
	•	Think of this as querying a catalog of objects.<br/>
GET https://nominatim.openstreetmap.org/search?q=Statue+of+Liberty&format=json<br/>
<br/>
Simulate a document search<br/>
	•	Useful for document-based retrieval like CDF events or files.<br/>
	•	Great for agent retrieval demos with RAG.<br/>
GET http://export.arxiv.org/api/query?search_query=all:ai&start=0&max_results=2<br/>

In [0]:
#Let's create some sample functions to test our endpoints. We can call these just to get a sense of how they behave and what kind of input parameters we'll need to inject.

import requests

def fetch_weather(latitude: float, longitude: float):
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m",
    }
    response = requests.get(url, params=params)
    return response.json()

def search_assets(query: str):
    url = "https://nominatim.openstreetmap.org/search"
    params = {"q": query, "format": "json"}
    response = requests.get(url, params=params)
    return response.json()

def search_arxiv(keywords: str, max_results: int = 3):
    url = "http://export.arxiv.org/api/query"
    params = {
        "search_query": f"all:{keywords}",
        "start": 0,
        "max_results": max_results
    }
    response = requests.get(url, params=params)
    return response.text  # XML format, you can parse with `feedparser`

In [0]:
#Let's do a quick preview of our data. The easiest way to do this is to throw it into a dataframe. We're just looking for a successful call and loose schema.
# We can do the same for any of our API calls

import pandas as pd

#Let's get a sample from Calgary
pd.DataFrame(fetch_weather(51.0478, 114.0593))

## Using LangChain's tools library to create tool stubs
This is a pretty easy approach, using a declarative approach and LangChain's tool library. All you need to do is register a function with a name and detailed description that will help the agent understand what each tool does.

In [0]:
#LangChain provides a quick and easy way to log functions as tools. This is an 'easy button' but we lose a degree of control in terms of behaviour management. Nonetheless, it's a pretty easy way to get going if you just want to do something simple.

#Although we probably won't be using this set of tools, it's good to know that it's an option for rapid development or prototyping.

from langchain.tools import Tool

tools = [
    Tool.from_function(fetch_weather, name="FetchWeather", description="Fetch hourly weather data."),
    Tool.from_function(search_assets, name="SearchAssets", description="Search for asset-like locations."),
    Tool.from_function(search_arxiv, name="SearchArxiv", description="Search research documents."),
]